### Observations
Observation 1 - Factors other than the amount spent per student is contributing to higher performance. There is a negative    correlation between spending per student and performance.

Observation 2 - Charter schools out performed District schools even when you factor of amount spent per student.

Observation 3 - Smaller size student schools performed better than larger size schools but Charter schools never the less performed better regardless of size.

Observation 4 – No socioeconomic or crime data where included which can add to the understanding of lower student performance in District schools.


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
schoolfile = "Resources/schools_complete.csv"
studentfile = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school = pd.read_csv(schoolfile)
student = pd.read_csv(studentfile)


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#calculate passing grades for math,reading and for overallall
math = (student["math_score"] >= 70).value_counts()[True]
p_math= round((math/len(student)*100),2)

read = (student["reading_score"] >= 70).value_counts()[True]
p_read = round((read/len(student)*100),2)

p_all = round((p_math + p_read)/2,2)

#Create dataframe and format
dist_sum = pd.DataFrame({'Total Schools': [round(len(school),2)], 
                                 'Total Students': [round(len(student),2)], 
                                 'Total Budget':[school['budget'].sum()], 
                                'Average Math score':[round(student["math_score"].mean(),2)], 
                                'Average Reading score':[round(student["reading_score"].mean(),2)],
                                     '% Passing Math':[round((math/len(student))*100,2)],
                                     '% Passing Reading':[round((read/len(student))*100,2)],
                                     '% Overall Passing Rate':[(p_all)]
                                     })

#Formatting
dist_sum['Total Budget'] = dist_sum['Total Budget'].map('$ {:,.2f}'.format)
dist_sum['Total Students'] = dist_sum['Total Students'].map('{:,.2f}'.format)

#print results with labels
dist_sum.style.set_caption('District Summary')

,Total Schools,Total Students,Total Budget,Average Math score,Average Reading score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170.00","$ 24,649,428.00",78.99,81.88,74.98,85.81,80.4


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
#School Name, type, total students, school budget, student budget
df_school=school.groupby(["school_name","type"])[["size","budget"]].sum().reset_index()
df_school['Per Student Budget'] = df_school['budget']/df_school['size']

#School average math and reading score
df_student=student.groupby(["school_name"])[["math_score","reading_score"]].mean().reset_index()
df_std_cnt=student.groupby(["school_name"])[["math_score"]].count().reset_index()


#count passing math
std_math_pass = student[student["math_score"] >= 70].groupby(["school_name"])["math_score"].count().reset_index()

df_student['% Passing Math'] = round((std_math_pass['math_score']/df_std_cnt['math_score']*100),2)

#count passing reading
std_read_pass = student[student["reading_score"] >= 70].groupby(["school_name"])["reading_score"].count().reset_index()

df_student['% Passing Reading'] = round((std_read_pass['reading_score']/df_std_cnt['math_score']*100),2)

#overall passing rate
df_student['% Overall Passing Rate'] = round((df_student['% Passing Math'] + df_student['% Passing Reading'])/2,2)


#Merge school and student dataframes
df_all_data=df_school.merge(df_student, on='school_name')

#Create new dataframe called school
schools = pd.DataFrame(df_all_data)

#Label column on new dataframe

schools.columns = ['School Name','School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']
schools['Total School Budget'] = schools['Total School Budget'].map('{:,.0f}'.format)
#schools['Total Students'] = schools['Total Students'].map('{:,.0f}'.format)
schools['Average Math Score']=schools['Average Math Score'].apply(lambda x:round(x,2))
schools['Average Reading Score']=schools['Average Reading Score'].apply(lambda x:round(x,2))


schools.style.set_caption('School Summary')


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,4976,"3,124,928",628,77.05,81.03,66.68,81.93,74.31
1,Cabrera High School,Charter,1858,"1,081,356",582,83.06,83.98,94.13,97.04,95.58
2,Figueroa High School,District,2949,"1,884,411",639,76.71,81.16,65.99,80.74,73.36
3,Ford High School,District,2739,"1,763,916",644,77.1,80.75,68.31,79.3,73.81
4,Griffin High School,Charter,1468,"917,500",625,83.35,83.82,93.39,97.14,95.26
5,Hernandez High School,District,4635,"3,022,020",652,77.29,80.93,66.75,80.86,73.81
6,Holden High School,Charter,427,"248,087",581,83.8,83.81,92.51,96.25,94.38
7,Huang High School,District,2917,"1,910,635",655,76.63,81.18,65.68,81.32,73.5
8,Johnson High School,District,4761,"3,094,650",650,77.07,80.97,66.06,81.22,73.64
9,Pena High School,Charter,962,"585,858",609,83.84,84.04,94.59,95.95,95.27


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
#Print with label top 5 schools
schools.nlargest(5,"% Overall Passing Rate").style.set_caption("Top Performing Schools (By Passing Rate)")

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
1,Cabrera High School,Charter,1858,"1,081,356",582,83.06,83.98,94.13,97.04,95.58
12,Thomas High School,Charter,1635,"1,043,130",638,83.42,83.85,93.27,97.31,95.29
9,Pena High School,Charter,962,"585,858",609,83.84,84.04,94.59,95.95,95.27
4,Griffin High School,Charter,1468,"917,500",625,83.35,83.82,93.39,97.14,95.26
13,Wilson High School,Charter,2283,"1,319,574",578,83.27,83.99,93.87,96.54,95.21


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
#Print with label lowest 5 schools
schools.nsmallest(5,"% Overall Passing Rate").style.set_caption("Bottom Performing Schools (By Passing Rate)")

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
10,Rodriguez High School,District,3999,"2,547,363",637,76.84,80.74,66.37,80.22,73.3
2,Figueroa High School,District,2949,"1,884,411",639,76.71,81.16,65.99,80.74,73.36
7,Huang High School,District,2917,"1,910,635",655,76.63,81.18,65.68,81.32,73.5
8,Johnson High School,District,4761,"3,094,650",650,77.07,80.97,66.06,81.22,73.64
3,Ford High School,District,2739,"1,763,916",644,77.1,80.75,68.31,79.3,73.81


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#Scores by Math and Grade
a_grade = student[student["grade"] == "9th"].groupby("school_name")["math_score"].mean()
b_grade = student[student["grade"] == "10th"].groupby("school_name")["math_score"].mean()
c_grade = student[student["grade"] == "11th"].groupby("school_name")["math_score"].mean()
d_grade = student[student["grade"] == "12th"].groupby("school_name")["math_score"].mean()

# Combine the series into a dataframe
math_grade = pd.DataFrame({"9th": round(a_grade,2), "10th": round(b_grade,2), "11th": round(c_grade,2), "12th": round(d_grade,2)})
math_grade = math_grade.rename_axis(['School Name'])

#Print results with label
math_grade.style.set_caption("Math Scores by Grade")

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.4,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#Scores by reading and Grade
a_grade = student[student["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
b_grade = student[student["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
c_grade = student[student["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
d_grade = student[student["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

# Combine the series into a dataframe
read_grade = pd.DataFrame({"9th": round(a_grade,2), "10th": round(b_grade,2), "11th": round(c_grade,2), "12th": round(d_grade,2)})
read_grade = read_grade.rename_axis(['School Name'])

#Print results with label
math_grade.style.set_caption("Reading Scores by Grade")

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.4,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [8]:
#Testing combined dataframe for both scores
dfs = [math_grade, read_grade]

df_combined = pd.concat(
    [df.rename(columns=lambda x: x.zfill(4)) for df in dfs],
    keys=['Math Scores'.format(),'Reading Scores'.format()],
    axis=1
)

df_combined.style.set_caption("Math and Reading Scores by Grade - Testing New Table")

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
#Copy schools dataframe for modification
schools_by_spending = schools.copy()

# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#Create the bins and add column labels
spending_labels=pd.cut(schools_by_spending['Per Student Budget'], spending_bins, labels=group_names)

#Add column with labels to dataframe schools_by_spending
schools_by_spending["Spending Ranges (Per Student)"] = spending_labels

#Group by new spending ranges column
school_scores_grouped=schools_by_spending.groupby(["Spending Ranges (Per Student)"])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math','% Overall Passing Rate'].mean()

#Print results with label
school_scores_grouped.style.set_caption("Scores by School Spending")

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.9325,83.4525,96.61,93.46,95.035
$585-615,83.885,83.6,95.9,94.23,95.065
$615-645,81.8917,79.0783,86.1067,75.6683,80.8883
$645-675,81.0267,76.9967,81.1333,66.1633,73.65


In [10]:
#Testing numpy array method with different ranges

#Copy schools dataframe for modification
schools_by_spending = schools.copy()

#Using numpy array method
titles = np.array(['0 to 600', '600 to 625', '625 to 650', '650 to 675','675 to 700'])

schools_by_spending['Per Student Budget'] = pd.cut(schools_by_spending['Per Student Budget'], 5, labels=titles )


school_scores_grouped=schools_by_spending.groupby(['Per Student Budget'])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math','% Overall Passing Rate'].mean()

school_scores_grouped.style.set_caption("Scores by School Spending - Using Numpy array")


,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Per Student Budget,,,,,
0 to 600,83.9325,83.4525,96.61,93.46,95.035
600 to 625,83.73,83.36,95.85,93.87,94.86
625 to 650,84.04,83.84,95.95,94.59,95.27
650 to 675,82.12,79.474,87.468,77.14,82.304
675 to 700,80.9575,77.0225,80.675,66.7,73.69


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
schools_by_size = schools.copy()

# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


size_labels=pd.cut(schools_by_size['Total Students'], size_bins, labels=group_names)

schools_by_size["School Size"] = size_labels

school_size_grouped=schools_by_size.groupby(["School Size"])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math','% Overall Passing Rate'].mean()

school_size_grouped.style.set_caption("Scores by School Size")


,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.925,83.82,96.1,93.55,94.825
Medium (1000-2000),83.866,83.374,96.79,93.598,95.192
Large (2000-5000),81.3438,77.745,82.7662,69.9638,76.3675


In [12]:
#Testing numpy array method with different ranges

#Copy schools dataframe for modification
schools_by_size = schools.copy()

#Using numpy array method
titles = np.array(['0 to 1000', '1000 to 2000', '2000 to 5000'])

schools_by_size['Total Students'] = pd.cut(schools_by_size['Total Students'], 3, labels=titles )


school_size_grouped=schools_by_size.groupby(['Total Students'])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math','% Overall Passing Rate'].mean()

school_size_grouped.style.set_caption("Scores by School Size - Using Numpy array")

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Total Students,,,,,
0 to 1000,83.8829,83.5014,96.5929,93.5843,95.0871
1000 to 2000,81.77,78.4275,84.475,73.4625,78.97
2000 to 5000,80.9175,77.0625,81.0575,66.465,73.765


## Scores by School Type

* Perform the same operations as above, based on school type.

In [13]:
schools_by_type = schools.copy()

schools_by_type_grouped=schools_by_type.groupby(['School Type'])['Average Reading Score', 'Average Math Score', '% Passing Reading', '% Passing Math','% Overall Passing Rate'].mean()

schools_by_type_grouped.style.set_caption("Scores by School Type")



,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Type,,,,,
Charter,83.8963,83.4725,96.5862,93.62,95.1025
District,80.9657,76.9557,80.7986,66.5486,73.6757
